<div style="background-color: #1e1e1e; color: #e0e0e0; padding: 20px; border-radius: 15px; border: 1px solid #333; box-shadow: 0 4px 8px rgba(0,0,0,0.5);">
    
<div style="text-align: center; border-bottom: 2px solid #00d4ff; padding-bottom: 15px; margin-bottom: 20px;">
        <h1 style="color: #00d4ff; font-family: sans-serif; font-weight: 700; letter-spacing: 1px;">
            🛒 The Apriori Algorithm
        </h1>
        <p style="font-size: 1.1em; color: #b0bec5;">
            Uncovering hidden patterns in transaction data
        </p>
    </div>

<p style="font-family: sans-serif; line-height: 1.6; font-size: 16px;">
        <b>Association Rule Learning</b> is the technique behind recommendation engines (e.g., "Frequently Bought Together"). The <b>Apriori Algorithm</b> is the classic method used to identify these relationships in large transactional datasets.
    </p>

<h2 style="color: #b388ff; margin-top: 30px;">1. The Intuition</h2>
    <div style="background-color: #2d2d2d; border-left: 5px solid #b388ff; padding: 15px; border-radius: 5px;">
        <p style="margin: 0;">
            <b>The "Bottom-Up" Strategy:</b><br>
            Apriori doesn't look at everything at once. It starts small:<br>
            1. Find individual items that are popular.<br>
            2. Combine them into pairs. Are the pairs popular?<br>
            3. Combine pairs into triplets. Are the triplets popular?<br>
            4. Stop when you can't go any further.
        </p>
    </div>

<h2 style="color: #69f0ae; margin-top: 30px;">2. The Three Pillars (Metrics)</h2>
    <p>To filter the noise from the signal, we use three key metrics.</p>
    
<table style="width:100%; border-collapse: separate; border-spacing: 0 10px;">
        <thead style="background-color: #333; color: #69f0ae;">
            <tr>
                <th style="padding: 10px; text-align: left; border-radius: 5px 0 0 5px;">Metric</th>
                <th style="padding: 10px; text-align: left;">Definition</th>
                <th style="padding: 10px; text-align: left; border-radius: 0 5px 5px 0;">Math Logic</th>
            </tr>
        </thead>
        <tbody>
            <tr style="background-color: #2d2d2d;">
                <td style="padding: 12px; font-weight: bold; color: #ffffff;">📊 Support</td>
                <td style="padding: 12px;">How popular is item A? (Frequency)</td>
                <td style="padding: 12px; font-family: monospace; color: #90caf9;">Freq(A) / Total</td>
            </tr>
            <tr style="background-color: #2d2d2d;">
                <td style="padding: 12px; font-weight: bold; color: #ffffff;">🎯 Confidence</td>
                <td style="padding: 12px;">If I buy A, how likely am I to buy B?</td>
                <td style="padding: 12px; font-family: monospace; color: #90caf9;">Support(A&B) / Support(A)</td>
            </tr>
            <tr style="background-color: #2d2d2d;">
                <td style="padding: 12px; font-weight: bold; color: #ffffff;">🚀 Lift</td>
                <td style="padding: 12px;">Is the relationship real or just luck?</td>
                <td style="padding: 12px; font-family: monospace; color: #90caf9;">Conf(A->B) / Support(B)</td>
            </tr>
        </tbody>
    </table>

 <div style="margin-top: 10px; font-size: 0.9em; color: #ccc;">
        <ul>
            <li><b>Lift > 1:</b> Strong association (Buy A -> Likely to buy B)</li>
            <li><b>Lift = 1:</b> No association (Random chance)</li>
            <li><b>Lift < 1:</b> Negative association (Buy A -> Unlikely to buy B)</li>
        </ul>
    </div>

<h2 style="color: #ff8a80; margin-top: 30px;">3. The Golden Rule</h2>
    
 <div style="background-color: #3e2723; padding: 15px; border: 1px solid #ff8a80; border-radius: 8px; color: #ffccbc;">
        <h3 style="margin-top: 0; color: #ff8a80;">⚠️ The Apriori Principle</h3>
        <p>
            <i>"If an itemset is infrequent, then all of its supersets must also be infrequent."</i>
        </p>
        <p style="font-size: 0.9em;">
            <b>Translation:</b> If nobody buys <b>{Pickles}</b>, then it's impossible for many people to buy <b>{Pickles, Ice Cream}</b>. We can immediately throw away any combination containing Pickles. This saves huge amounts of computing power.
        </p>
    </div>

<h2 style="color: #fff; margin-top: 30px;">4. Pros & Cons</h2>
    <div style="display: flex; gap: 20px;">
        <div style="flex: 1; background-color: #1b5e20; padding: 15px; border-radius: 10px; color: #e8f5e9;">
            <h4 style="margin-top: 0; border-bottom: 1px solid rgba(255,255,255,0.3); padding-bottom: 5px;">✅ Advantages</h4>
            <ul style="padding-left: 20px; margin-bottom: 0;">
                <li>Easy to interpret results.</li>
                <li>Pruning reduces search space significantly.</li>
            </ul>
        </div>
        <div style="flex: 1; background-color: #b71c1c; padding: 15px; border-radius: 10px; color: #ffebee;">
            <h4 style="margin-top: 0; border-bottom: 1px solid rgba(255,255,255,0.3); padding-bottom: 5px;">❌ Disadvantages</h4>
            <ul style="padding-left: 20px; margin-bottom: 0;">
                <li>Slow on massive datasets.</li>
                <li>Needs multiple scans of the database.</li>
            </ul>
        </div>
    </div>


</div>


<div style="background-color: #1e1e1e; color: #cfd8dc; padding: 25px; border-radius: 15px; border: 1px solid #37474f; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; box-shadow: 0 10px 20px rgba(0,0,0,0.5);">
    
<div style="text-align: center; border-bottom: 2px solid #7c4dff; padding-bottom: 20px; margin-bottom: 30px;">
        <h1 style="color: #7c4dff; font-weight: 800; margin-bottom: 5px;">
             Dynamic Hashing & Pruning (DHP)
        </h1>
        <p style="font-size: 1.1em; color: #b0bec5; letter-spacing: 1px;">
            Optimizing the <span style="color: #ff4081; font-weight: bold;">$C_2$</span> Bottleneck
        </p>
    </div>

<h2 style="color: #00e5ff; border-left: 4px solid #00e5ff; padding-left: 15px;">1. The Bottleneck: Why optimize $C_2$?</h2>
    <p>
        In standard Apriori, the most expensive step is generating the candidate pairs ($C_2$). Even if you filter out infrequent individual items, the number of possible pairs explodes combinatorially.
    </p>
    <div style="background-color: #263238; padding: 15px; border-radius: 8px; text-align: center; border: 1px dashed #546e7a;">
        <p style="margin: 0; color: #eceff1;">
            If you have 1,000 frequent items, you must check:<br>
            $$ \frac{1000 \times 999}{2} \approx 500,000 \text{ pairs} $$
        </p>
    </div>

 <h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">2. The Hashing Strategy</h2>
    <p>
        The core idea is to <b>"peek ahead"</b>. While we scan the database to count individual items ($L_1$), we can simultaneously gather rough data about pairs using a <b>Hash Table</b>.
    </p>

<ul style="list-style-type: none; padding: 0;">
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">1.</span>
            <div><b>Scan:</b> As we read a transaction, generate all possible 2-item subsets.</div>
        </li>
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">2.</span>
            <div><b>Hash:</b> Apply a function to map the pair to a specific bucket index.</div>
        </li>
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">3.</span>
            <div><b>Count:</b> Increment the counter for that bucket.</div>
        </li>
    </ul>

<h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">3. Visual Example</h2>
    
 <p>Assume <b>Min_Support = 3</b> and Hash Function: $h(x,y) = (x \cdot 10 + y) \pmod 7$.</p>

 <table style="width:100%; border-collapse: separate; border-spacing: 0; border: 1px solid #455a64; border-radius: 8px; overflow: hidden;">
        <thead style="background-color: #37474f; color: #00e5ff;">
            <tr>
                <th style="padding: 12px;">Transaction</th>
                <th style="padding: 12px;">Generated Pair</th>
                <th style="padding: 12px;">Hash Calc</th>
                <th style="padding: 12px;">Bucket Index</th>
            </tr>
        </thead>
        <tbody style="color: #b0bec5;">
            <tr style="background-color: #263238;">
                <td rowspan="3" style="padding: 12px; border-bottom: 1px solid #455a64; vertical-align: middle; font-weight: bold;">{1, 2, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{1, 2}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(12) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 5</td>
            </tr>
            <tr style="background-color: #263238;">
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{1, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(13) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 6</td>
            </tr>
            <tr style="background-color: #263238;">
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{2, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(23) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 2</td>
            </tr>
        </tbody>
    </table>

<div style="margin-top: 30px; padding: 20px; background-color: #212121; border-left: 5px solid #ff4081; border-radius: 0 10px 10px 0;">
        <h3 style="margin-top: 0; color: #ff4081;">✂️ The Pruning Rule</h3>
        <p>
            When creating $C_2$, we check the bucket count for every candidate pair.
        </p>
        <p style="font-size: 1.1em; font-weight: bold; color: #fff;">
            IF <code>Bucket_Count < Min_Support</code> THEN Discard Pair.
        </p>
        <p style="font-size: 0.9em; color: #90caf9;">
            <i>Why? Because even if items A and B are individually frequent, if their combined bucket count is low, they mathematically <b>cannot</b> be frequent together.</i>
        </p>
    </div>

<h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">4. The "Collision" Catch</h2>
    <p>
        Sometimes, two infrequent pairs land in the same bucket, making the bucket count artificially high (False Positive).
    </p>
    
<div style="display: flex; gap: 15px; margin-top: 15px;">
        <div style="flex: 1; background-color: #1b5e20; padding: 15px; border-radius: 8px; color: #a5d6a7;">
            <b>Low Bucket Count:</b><br>
            100% Safe to Prune. <br>The pair is definitely infrequent.
        </div>
        <div style="flex: 1; background-color: #b71c1c; padding: 15px; border-radius: 8px; color: #ffcdd2;">
            <b>High Bucket Count:</b><br>
            Might be frequent, or might be a collision.<br> We must keep it and check later.
        </div>
    </div>

</div>
